# Import other python packages 

The NWHD Connector use features of the [PyDEEN Package](https://pypi.org/project/pydeen/). For Data Science activities the pandas library is used to return Dataframe objects.


In [ ]:
# import libs 
import pandas as pd
from pynwhd import NWHDConnector

# Configure the connector
Maintain your configuration here.

In [ ]:
# configure backend and parameters
sid       = "D01"
desc      = "SAP Dev"
client    = "100"
url       = "http://saphost"
api_path  = "/nwhd_rest/v1" 
source    = "00505699C4E91EDDA6DDC77FA530E9ED"

date_from = "20230101"
date_to   = "20230115"
max_rows  = 100000

# create a connector object in script mode and configure

At the first start you have to enter the authentification details via menu. You can save and reuse it for next calls. The information will be safed encoded in the current path.

A list of all usable source systems and the later required source GUID is displayed. The set_source command (check the GUID) will display a list of available numeric value keys.

In [ ]:
# use connector in scripting mode and configure
connector = NWHDConnector(sid, url, client, desc, api_path)
connector.set_date_interval(date_from, date_to)
connector.set_max_rows(max_rows)
connector.set_source(source)

# Get prepared numeric values and merge

In [ ]:
# get prepared timeseries data for single values
df1 = connector.get_df_numeric("UserMemory:SessionMemory:Sum")
df1.plot()

df2 = connector.get_df_numeric("BALog:LastHour:AllTypeMsgError")
df2.plot()

df3 = connector.get_df_numeric("BALog:DB:Count")
df3.plot()

# merge it
# join dataframes via index
df_join = pd.concat([df1, df2, df3], axis=1)
df_join
df_join.plot()

# Select multiple numeric values

In [ ]:
# get data as joined dataframe via multiple value columns
set = [
    "UserMemory:SessionMemory:Sum",
    "UserMemory:SessionMemory:Max",
    "UserMemory:SessionMemory:Avg",
    "Users:Current:LoggedOn",
    "Users:Current:Modes",
    "Users:Current:Sessions"
]

df_set = connector.get_df_numeric_multiple(set)
df_set.plot()
df_set.dtypes
df_set.index

# Expert mode: raw extraction and preparation

In [ ]:
# manual extraction & manipulation
df_usage = connector.get_numeric_values_as_dataframe(source, "WP", "TypeDIA", "Usage", date_from=date_from, date_to=date_to, max_rows=max_rows, prepare=False)

df_usage['TIMESTAMP'] = pd.to_datetime(df_usage['TIMESTAMP'], format='%Y-%m-%d %H:%M')
df_usage.set_index(pd.DatetimeIndex(df_usage.TIMESTAMP), inplace=True)

df_usage.drop('TIMESTAMP', axis=1, inplace=True)
df_usage.rename(columns={'VALUE': 'WP:TypeDIA:Usage'}, inplace=True)

df_usage.plot()
df_usage.dtypes
df_usage.index